In [15]:
import sys
sys.path.insert(1, 'F:/Dev/LMEmbeddingAnalysis/') # Change to what you need

In [13]:
import json
import torch
from torch.nn import Embedding

from utils.constants import HOME_DIR

In [14]:
model_names = [
    "t5-v1_1-small",
    "t5-v1_1-base",
    "t5-v1_1-large",
    "t5-v1_1-xl",
    "t5-v1_1-xxl",
]

model_paths = [ # the location of the saved models on my machine. You could also make this 🤗 ids, e.g. google/t5-v1_1-small. You may need to change more of the code, though.
    "F:/Models/google/" + model_name + "" for model_name in model_names
]

model_filenames = [ # this is the simplest way to deal with multipart files when I just need the first one (the embedding is just in the first one)
    "pytorch_model.bin",
    "pytorch_model.bin",
    "pytorch_model.bin",
    "pytorch_model.bin",
    "pytorch_model.bin",
]

In [7]:
weights = torch.load(model_paths[0] + "/" + model_filenames[0], map_location='cpu')
weight_key = list(weights.keys())[1] # May not be 1, may be 0 or something else
print(weight_key) # Make sure this is what you want! It'll probably say "embed" somewhere, and is different for each model suite, and even sometimes for models within the model suite

encoder.embed_tokens.weight


In [10]:
for model_name, model_path, model_filename in zip(model_names, model_paths, model_filenames):
    if model_name in [ # Easy way to select precisely the ones you need. Some files are weird and have different weight keys even for the same model suite
        "t5-v1_1-base",
        "t5-v1_1-large",
        "t5-v1_1-xl",
        "t5-v1_1-xxl",
    ]:
        continue
    embedding_filename = f'../embeddings/{model_name}.pth'

    config_filename = model_path + '/config.json'
    with open(config_filename) as file:
        config_dict = json.loads(file.read())
        vocab_side = config_dict['vocab_size']
        d_model = config_dict['d_model']
        print(vocab_side,d_model)
        embedding_layer = Embedding(vocab_side, d_model)
        print(embedding_layer(torch.tensor(0)).size())
        
    weights = torch.load(model_path + "/" + model_filename, map_location='cpu')  # Adjust path as needed
    # print(weights.keys())
    print(weights['encoder.embed_tokens.weight'])
    embedding_layer.load_state_dict({"weight":weights['encoder.embed_tokens.weight']}) 
    # print(embedding_layer.weight[0])

    torch.save(embedding_layer, embedding_filename, pickle_protocol=4)

32128 512
torch.Size([512])
tensor([[-1.2188e+00, -4.3750e+00, -7.9062e+00,  ...,  2.0469e+00,
          3.0938e+00, -6.5938e+00],
        [-1.4438e+01,  8.1250e+00, -1.1719e+00,  ...,  1.1562e+01,
          4.8438e+00,  9.1000e+01],
        [ 8.1250e+00,  3.6250e+00, -1.9453e+00,  ..., -4.6250e+00,
          1.3125e+01,  2.1375e+01],
        ...,
        [-4.5117e-01, -3.3594e-01, -3.8867e-01,  ..., -2.0996e-01,
         -2.0000e+00, -9.1406e-01],
        [-1.0234e+00, -8.0859e-01,  4.3555e-01,  ..., -5.9326e-02,
         -9.2188e-01, -9.2969e-01],
        [ 1.0078e+00,  1.5234e-01, -2.4902e-01,  ..., -1.8555e-01,
         -2.7148e-01,  1.7969e+00]])
